In [73]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes
import Data.Char
import Control.Applicative

<h4>Composition as Functor, Applicative and Monad</h4>

In [2]:
cap :: [Char] -> [Char]
cap = map toUpper

rev :: [Char] -> [Char]
rev = reverse

In [3]:
composed :: [Char] -> [Char]
composed = rev . cap

In [5]:
fmapped :: [Char] -> [Char]
fmapped = fmap rev cap

In [7]:
tupled :: [Char] -> ([Char], [Char])
tupled = (,) <$> rev <*> cap

In [20]:
tupledMonad :: [Char] -> ([Char], [Char])
tupledMonad = do
    a <- rev
    b <- cap
    return (a, b)

In [21]:
tupledMonad "Julie"

("eiluJ","JULIE")

<h4>Function applicative</h4>

In [64]:
newtype HumanName = HumanName String deriving (Eq, Show)

newtype DogName = DogName String deriving (Eq, Show)

newtype Address = Address String deriving (Eq, Show)

In [66]:
data Person = 
    Person {
         humanName :: HumanName
        ,dogName :: DogName
        ,address :: Address
    } deriving (Eq, Show)
    
data Dog =
    Dog {
         dogsName :: DogName
        ,dogsAddress :: Address
    } deriving (Eq, Show)

In [67]:
p1 = Person (HumanName "Rohit") (DogName "D1") (Address "NYC")
p2 = Person (HumanName "Rashi") (DogName "D2") (Address "NY")

In [85]:
getDog :: Person -> Dog
getDog p = Dog (dogName p) (address p)

getDogR :: Person -> Dog
getDogR = Dog <$> dogName <*> address

getDogR' :: Person -> Dog
getDogR' = liftA2 Dog dogName address

<h4>Reading Comprehension Exercise</h4>

In [86]:
myLiftA2 :: Applicative f => (a -> b -> c) -> f a -> f b -> f c
myLiftA2 f a b = f <$> a <*> b

<h4>Function Monad</h4>

In [96]:
foo :: (Functor f, Num a) => f a -> f a
foo r = fmap (+1) r

bar :: Foldable f => t -> f a -> (t, Int)
bar r t = (r, length t)

In [97]:
froot :: Num a => [a] -> ([a], Int)
froot r = (map (+1) r, length r)

In [105]:
barPlus r = (foo r, length r)

frooty :: Num a => [a] -> ([a], Int)
frooty r = bar (foo r) r

frooty' :: Num a => [a] -> ([a], Int)
frooty' = \r -> bar (foo r) r

In [112]:
fooBind :: (r -> a) -> (a -> r -> b) -> (r -> b)
fooBind m k = \r -> k (m r) r

In [113]:
:t fooBind

fooBind :: forall r a b. (r -> a) -> (a -> r -> b) -> r -> b

In [114]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

In [107]:
froot [1,2,3] == barPlus [1,2,3]
barPlus [1,2,3] == frooty [1,2,3]
frooty [1,2,3] == frooty' [1,2,3]

True

True

True

<h4>Custom Reader</h4>

In [27]:
newtype Reader r a = Reader (r -> a)

In [29]:
runReader :: Reader r a -> (r -> a)
runReader (Reader x) = x

In [63]:
ask :: Reader a a
ask = Reader id

In [40]:
instance Functor (Reader r) where
    fmap f (Reader x) = Reader (f . x)

In [41]:
rdr :: Reader String Int
rdr = Reader (\x -> length x)

rdrFMapped = fmap (\x -> x*x) rdr

In [43]:
runReader rdrFMapped "Rohit"

25

In [92]:
{-# LANGUAGE InstanceSigs #-}

instance Applicative (Reader r) where
    pure :: a -> Reader r a
    pure x = Reader (const x)
    
    (<*>) :: Reader r (a -> b) -> Reader r a -> Reader r b
    (<*>) (Reader rab) (Reader ra) = Reader $ \r -> rab r (ra r)

{-- Applicative f =>
f (b -> c) -> f b -> f c

f ~ Reader r

(Reader r) (b -> c) -> (Reader r) b -> (Reader r) c
--}

In [129]:
instance Monad (Reader r) where
    return = pure

{--
    (r -> a) -> (a -> r -> b) -> (r -> b)
--}
    (>>=) :: Reader r a -> (a -> Reader r b) -> Reader r b
    (>>=) (Reader ra) aRb = Reader $ \r -> runReader (aRb (ra r)) r

In [130]:
getDogRM :: Reader Person Dog
getDogRM = Reader $ Dog <$> dogName <*> address

In [132]:
runReader getDogRM p1

Dog {dogsName = DogName "D1", dogsAddress = Address "NYC"}

<h4>Exercise</h4>

In [133]:
import Control.Applicative
import Data.Maybe

In [134]:
x = [1..3]
y = [4..6]
z = [7..9]

In [139]:
xs :: Maybe Integer
xs = lookup 3 (zip x y)

ys :: Maybe Integer
ys = lookup 6 (zip y z)

zs :: Maybe Integer
zs = lookup 4 (zip x y)

In [148]:
z' :: Integer -> Maybe Integer
z' n = lookup n (zip x z)

In [154]:
x1 :: Maybe (Integer, Integer)
x1 = (,) <$> xs <*> ys

x2 :: Maybe (Integer, Integer)
x2 = (,) <$> ys <*> zs

In [156]:
x3 :: Integer -> (Maybe Integer, Maybe Integer)
x3 n = (z' n, z' n)

In [160]:
summed :: Num c => (c, c) -> c
summed = uncurry (+)

In [167]:
bolt :: Integer -> Bool
bolt x = (x > 3) && (x < 8)

In [170]:
main :: IO ()
main = do
    print $ sequenceA [Just 3, Just 2, Just 1]
    print $ sequenceA [x, y]
    print $ [xs, ys]
    print $ summed <$> ((,) <$> xs <*> ys)
    print $ fmap summed ((,) <$> xs <*> zs)
    print $ bolt 7
    print $ fmap bolt z

In [171]:
main

Just [3,2,1]
[[1,4],[1,5],[1,6],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6]]
[Just 6,Just 9]
Just 15
Nothing
True
[True,False,False]

In [182]:
sequA :: Integral a => a -> [Bool]
sequA = sequenceA [(>3), (<8), even]

In [184]:
s' = summed <$> ((,) <$> xs <*> ys)